## Preprocessing

Include any code needed to, for example, create CSV table, below.

## Defining paths 

Next, define the following paths/names,

1. `data_location`, which can be a path or `glob` search query,
2. `name_re`, which is a regular expression for extracting the table name from the table path, and 
3. `db_path`, which is the path to the destination db.

In [1]:
!ls -al

total 144
drwxr-xr-x@ 12 toddiverson  staff    384 Mar 22 07:07 .
drwx------@ 22 toddiverson  staff    704 Mar 21 09:53 ..
drwxr-xr-x  13 toddiverson  staff    416 Mar 22 07:24 .git
drwxr-xr-x   4 toddiverson  staff    128 Mar 22 07:10 .ipynb_checkpoints
-rw-r--r--   1 toddiverson  staff     46 Mar 22 07:25 README.md
-rw-r--r--   1 toddiverson  staff    292 Mar 22 07:22 Untitled.ipynb
-rw-r--r--@  1 toddiverson  staff  32988 Mar 21 09:47 __construct_example_database.ipynb
-rw-r--r--   1 toddiverson  staff  11935 Mar 22 07:30 concat_str_with_python_and_polars.ipynb
-rw-------@  1 toddiverson  staff     50 Mar 22 07:09 emails.csv
-rw-------@  1 toddiverson  staff    750 Mar 22 07:09 emails.jmp
-rw-------@  1 toddiverson  staff    105 Mar 22 07:08 form_output.csv
-rw-------@  1 toddiverson  staff    894 Mar 22 07:08 form_output.jmp


In [2]:
data_location = './*.csv'
name_re = r'./(.*).csv'
db_path = 'sqlite:///example.db'

## Create the db

Run the following cell to create the database.

In [3]:
!pip install sqlalchemy-utils

In [4]:
import pandas as pd
from glob import glob
from dfply import *
import re
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import Integer, Float, String, DateTime

DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}



def get_sql_types(df):
    sql_type = lambda dtype: DTYPES_TO_SQLALCHEMY_TYPES[dtype.kind] 
    cols_and_dtypes = lambda df: zip(df.columns, df.dtypes)
    return {col:sql_type(dtype) 
            for col, dtype in cols_and_dtypes(df)}

paths = glob(data_location)
paths

['./emails.csv', './form_output.csv']

In [5]:
name_re = re.compile(name_re)
names = [name_re.match(path).group(1) for path in paths]
names

['emails', 'form_output']

In [6]:
tables = {name: pd.read_csv(path)  >> mutate(RowID = X.index) for name, path in zip(names, paths)}
types = {name:get_sql_types(t) for name, t in tables.items()}

In [7]:
tables

{'emails':   Team            Email  RowID
 0  N01  xyz@stjohns.edu      0
 1  U01   abc@winona.edu      1,
 'form_output':   Team  Score     Comments  RowID
 0  N01     78       Blah 1      0
 1  N01     73       Blah 2      1
 2  U01     54  Blah Blah 1      2
 3  U01     62  Blah Blah 2      3
 4  U01     68  Blah Blah 3      4}

In [8]:
types

{'emails': {'Team': sqlalchemy.sql.sqltypes.String,
  'Email': sqlalchemy.sql.sqltypes.String,
  'RowID': sqlalchemy.sql.sqltypes.Integer},
 'form_output': {'Team': sqlalchemy.sql.sqltypes.String,
  'Score': sqlalchemy.sql.sqltypes.Integer,
  'Comments': sqlalchemy.sql.sqltypes.String,
  'RowID': sqlalchemy.sql.sqltypes.Integer}}

In [10]:
engine = create_engine(db_path, echo=False)
# See https://stackoverflow.com/questions/6506578/how-to-create-a-new-database-using-sqlalchemy
if not database_exists(engine.url):
    create_database(engine.url)

for name in names:
    schema = pd.io.sql.get_schema(tables[name], # dataframe
                                  name, # name in SQL db
                                  keys='RowID', # primary key
                                  con=engine, # connection
                                  dtype=types[name] # SQL types
    )
    print(schema)
    engine.execute(schema)

for name in names:
    print("Loading table {0}".format(name))
    tables[name].to_sql(name, 
                        con=engine, 
                        dtype=types[name], 
                        index=False,
                        if_exists='append')


CREATE TABLE emails (
	"Team" VARCHAR, 
	"Email" VARCHAR, 
	"RowID" INTEGER NOT NULL, 
	CONSTRAINT emails_pk PRIMARY KEY ("RowID")
)



CREATE TABLE form_output (
	"Team" VARCHAR, 
	"Score" INTEGER, 
	"Comments" VARCHAR, 
	"RowID" INTEGER NOT NULL, 
	CONSTRAINT form_output_pk PRIMARY KEY ("RowID")
)


Loading table emails
Loading table form_output
